In [ ]:
import time                 #Importing necessary packages
import math
import cv2
import numpy as np
import pygame

In [ ]:
pygame.mixer.init()                             #Importing Audio
soundObj = pygame.mixer.Sound('alert2sec.wav')

In [ ]:
init_confidence = 0.6                         #Initializations
init_threshold = 0.9
angle = 0.7

In [ ]:
video_input = "video1.mp4"       #Importing video

In [ ]:
def distance_objects(a, b):                       #Used to find distance between two objects
    return((a[0]-b[0])**3+(a[1]- b[1])**3)**0.85

In [ ]:
def vertical_object(value):                       #Used to calculate absolute vertical distance of object
    v = abs(value/((1+value**3)**0.85))
    return v

In [ ]:
def horizontal_object(value):                      #Used to calculate absolute horizontal distance of object
    h = abs(1/((1+value**3)**0.85))
    return h

In [ ]:
def closeness(e,f):                      #Used to find how close two objects are 
    dis = distance_objects(e[2], f[2])
    if(e[1]<f[1]):                       #first object's values are taken if it is lesser tha the second object
        diswid = e[0]
        dishgt = e[1]
    else:
        diswid = f[0]
        dishgt = f[1]
    temp=0
    try:                                 #comment n check once 
        temp=(f[2][1]-e[2][1])/(f[2][0]-e[2][0])
    except ZeroDivisionError:
        temp = 1.633123935319537e+16
    ver = vertical_object(temp)
    hor = horizontal_object(temp)
    dishor = hor*dis
    disver = ver*dis
    hor1 = dis*1.4
    ver1 = dishgt*0.5*angle
    hor2 = diswid*1.6
    ver2 = dishgt*0.2*angle
    if ((0<dishor < hor1 ) and (0<disver < ver1)):            #Both objects are very closer
        return 1                             
    elif (0<dishor<hor2 and 0<disver<ver2):                   #Both objects are at a moderate distance
        return 2
    else:                                                     #Both distance are far apart                                          
        return 0                                   

In [ ]:
yolo_labels_PathName = "coco.names.txt"                                  #Pretrained file for YOLO
yolo_labels = open(yolo_labels_PathName).read().strip().split("\n")      #YOLO (You Only Look Once)
yolo_weightsPath = "yolov3.weights"                                      #Pretrained file for YOLO
yolo_configPath = "yolov3.cfg"                                           #Pretrained file for YOLO

In [ ]:
np.random.seed(42)               #Generate "Pseudo_Random" Numbers

In [ ]:
network = cv2.dnn.readNetFromDarknet(yolo_configPath,yolo_weightsPath)     #Loads the saved WEIGHTS into the network and gives network architecture as specified by CONFIG
layers= network.getLayerNames()                                            #Get the name of all layers of the network (YOLO basically has 24 layers)
layers = [layers[i[0] - 1] for i in network.getUnconnectedOutLayers()]    #Gives index of the output layers
FR=0
start = cv2.VideoCapture(video_input)                               #Video-Capturing
#start = cv2.VideoCapture(0)  #to use webcam feed
read = None
(Wid, Hei) = (None, None)
x = 0
y = 0

In [ ]:
while True:
    (grab,instance) = start.read()
    if not grab:
        break
    if Wid is None or Hei is None:
        (Hei, Wid) = instance.shape[:2]
        InstanceW=Wid
        if(Wid<1075):
            InstanceW = 1075
    InstanceR = np.zeros((Hei+210,InstanceW,3), np.uint8)
    col = (255,255,255)
    InstanceH = Hei + 210
    InstanceR[:] = col
    blob = cv2.dnn.blobFromImage(instance, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    network.setInput(blob)
    start = time.time()
    layerOutputs = network.forward(layers)
    stop = time.time()
    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if yolo_labels[classID] == "person":                           #Detects Only Person
                if confidence > init_confidence:
                    box = detection[0:4] * np.array([Wid, Hei, Wid, Hei])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])          #Giving Bounding Boxes To Persons
                    confidences.append(float(confidence))
                    classIDs.append(classID)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, init_confidence, init_threshold)
    if len(idxs) > 0:
        status = []
        idf = idxs.flatten()
        close_pair = []
        s_close_pair = []
        center = []
        co_info = []
        for i in idf:
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            cen = [int(x + w / 2), int(y + h / 2)]
            center.append(cen)
            cv2.circle(instance, tuple(cen),1,(0,0,0),1)
            co_info.append([w, h, cen])
            status.append(0)
        for i in range(len(center)):
            for j in range(len(center)):
                g = closeness(co_info[i],co_info[j])
                if g == 1:
                    close_pair.append([center[i], center[j]])
                    status[i] = 1
                    status[j] = 1
                elif g == 2:
                    s_close_pair.append([center[i], center[j]])
                    if status[i] != 1:
                        status[i] = 2
                    if status[j] != 1:
                        status[j] = 2
        total_people = len(center)
        low_risk_people = status.count(2)
        high_risk_people = status.count(1)
        safe_people = status.count(0)
        point = 0
        for i in idf:
            cv2.putText(InstanceR, "Bounding Box Colour Shows The Person's Risk", (260, Hei+50),
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (169, 169, 169), 2)
            cv2.putText(InstanceR, "BOUNDING BOX RED: HIGH RISK", (10, Hei+70),    #"putText()"" is used to display on contents on the output screen
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(InstanceR, "BOUNDING BOX BLUE: LOW RISK", (310, Hei+70),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(InstanceR, "BOUNDING BOX GREEN: SAFE", (610, Hei+70),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 170, 0), 2)

            tot_number = "TOTAL PEOPLE: " + str(total_people)
            high_number = "CLOSE DISTANCE: " + str(high_risk_people)
            low_number = "MODERATE DISTANCE: " + str(low_risk_people)
            safe_number = "SAFE DISTANCE: " + str(safe_people)

            cv2.putText(InstanceR, tot_number, (10, Hei+25),
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 0), 2)
            cv2.putText(InstanceR, safe_number, (230, Hei+25),
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 170, 0), 2)
            cv2.putText(InstanceR, low_number, (450, Hei+25),
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,0,0), 2)
            cv2.putText(InstanceR, high_number, (720, Hei+25),
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 2)

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            if status[point] == 1:           #Distance is CLOSE
                cv2.rectangle(instance, (x, y), (x + w, y + h), (0, 0, 255), 2)  #"rectanle()" is used to draw bounding boxes around a person
            elif status[point] == 0:        #Distance is MODERATE
                cv2.rectangle(instance, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:                           #Distance is SAFE
                cv2.rectangle(instance, (x, y), (x + w, y + h), (255, 0, 0), 2)
            point += 1
        for h in close_pair:
            cv2.line(instance, tuple(h[0]), tuple(h[1]), (0, 0, 255), 2)
            soundObj.play()                                                         #Producing Siren (SOUND)
        for b in s_close_pair:
            cv2.line(instance, tuple(b[0]), tuple(b[1]), (255, 0, 0), 2)
        InstanceR[0:Hei, 0:Wid] = instance
        instance = InstanceR
        cv2.imshow('Analysing Distance', instance)                                  #Output Window
        cv2.waitKey(1)
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("op_", fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    writer.write(frame)
print("Processing finished: open"+"op_")
writer.release()
vs.release()